In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
uid_train = pd.read_csv('/home/ss/Desktop/code/data/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('/home/ss/Desktop/code/data/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('/home/ss/Desktop/code/data/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('/home/ss/Desktop/code/data/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

voice_test = pd.read_csv('/home/ss/Desktop/code/data/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('/home/ss/Desktop/code/data/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('/home/ss/Desktop/code/data/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

#a=wa_test.isnull().sum()
#print a
#remove null uid
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})
uid_test.to_csv('/home/ss/Desktop/code/data/uid_test_b.txt',index=None)
#print uid_test.info()

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#lian he
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

In [7]:
#tong hua ji lu 
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()
voice_opp_head=voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()
voice_opp_len=voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)
voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)
voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)
#print voice_call_type

call_type    uid  voice_call_type_1  voice_call_type_2  voice_call_type_3  \
0          u0001               79.0                0.0                0.0   
1          u0002                2.0                0.0                0.0   
2          u0003               21.0                0.0                0.0   
3          u0004              218.0                6.0               30.0   
4          u0005              398.0                0.0                3.0   
5          u0006               39.0                5.0                0.0   
6          u0007               98.0                1.0                2.0   
7          u0008              234.0                0.0                0.0   
8          u0009               81.0                1.0               14.0   
9          u0010              121.0                0.0                9.0   
10         u0011              226.0                1.0                0.0   
11         u0012              247.0               15.0               40.0   

In [6]:
#duan xin jilu 
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index()
sms_opp_head=sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()
sms_opp_len=sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)
sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)
#print sms_opp_len

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
#fang wen ji lu
wa_name = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index()
    #fang wen ci shu 
visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()
    #zong shi chang
visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()
up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()
down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()
#print visit_cnt

        uid  wa_visit_cnt_std  wa_visit_cnt_max  wa_visit_cnt_min  \
0     u0001        290.475238            3747.0               1.0   
1     u0002          6.412734              24.0               1.0   
2     u0003         62.825353            1227.0               1.0   
3     u0004        109.912263            1568.0               1.0   
4     u0005         58.997481             620.0               1.0   
5     u0006         59.782370             671.0               1.0   
6     u0007         90.342535            1385.0               1.0   
7     u0008         75.292851             570.0               1.0   
8     u0009         72.075247             782.0               1.0   
9     u0010        312.227684            2445.0               1.0   
10    u0011         84.095565             792.0               1.0   
11    u0012         80.114817             916.0               1.0   
12    u0013         55.091364             721.0               1.0   
13    u0014         73.354374     

In [8]:
feature = [voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow,
           down_flow]

In [9]:
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')

In [10]:
test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')

In [11]:
train_feature.to_csv('/home/ss/Desktop/code/3/train_featureV1.csv',index=None)
test_feature.to_csv('/home/ss/Desktop/code/3/test_featureV1.csv',index=None)

In [12]:
train = pd.read_csv('/home/ss/Desktop/code/3/train_featureV1.csv')
test = pd.read_csv('/home/ss/Desktop/code/3/test_featureV1.csv')

In [13]:
dtrain = lgb.Dataset(train.drop(['uid','label'],axis=1),label=train.label)
dtest = lgb.Dataset(test.drop(['uid'],axis=1))

In [14]:
lgb_params =  {
    'boosting_type': 'gbdt',
    'objective': 'binary',
#    'metric': ('multi_logloss', 'multi_error'),
    #'metric_freq': 100,
#    'metric': {'auc'},
    'is_training_metric': False,
    'min_data_in_leaf': 12,
    'num_leaves': 63,
    'learning_rate': 0.06,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1
    'verbosity':-1,
#    'gpu_device_id':2,
#    'device':'gpu'
#    'lambda_l1': 0.001,
#    'skip_drop': 0.95,
#    'max_drop' : 10
    #'lambda_l2': 0.005
    #'num_threads': 18
 #   'seed': 7,
}

In [15]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)

    pre.preds=pre.preds.map(lambda x: 1 if x>=0.5 else 0)

    f1 = metrics.f1_score(pre.label,pre.preds)
    
    
    res = 0.6*auc +0.4*f1
    
    return 'res',res,True

In [16]:
lgb.cv(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=5,num_boost_round=10000,nfold=3,metrics=['evalMetric'])

[5]	cv_agg's res: 0.720755 + 0.0164622
[10]	cv_agg's res: 0.731429 + 0.00671881
[15]	cv_agg's res: 0.745871 + 0.00844311
[20]	cv_agg's res: 0.744047 + 0.00635199
[25]	cv_agg's res: 0.746811 + 0.00559785
[30]	cv_agg's res: 0.751973 + 0.0132078
[35]	cv_agg's res: 0.754736 + 0.011223
[40]	cv_agg's res: 0.758796 + 0.00922475
[45]	cv_agg's res: 0.761214 + 0.00958954
[50]	cv_agg's res: 0.757786 + 0.01138
[55]	cv_agg's res: 0.760347 + 0.00948082
[60]	cv_agg's res: 0.76146 + 0.0114929
[65]	cv_agg's res: 0.759059 + 0.0142529
[70]	cv_agg's res: 0.760621 + 0.0114323
[75]	cv_agg's res: 0.760695 + 0.0137636
[80]	cv_agg's res: 0.760064 + 0.0129591
[85]	cv_agg's res: 0.760476 + 0.0139708
[90]	cv_agg's res: 0.761912 + 0.0136609
[95]	cv_agg's res: 0.761569 + 0.0116365
[100]	cv_agg's res: 0.763526 + 0.0137156
[105]	cv_agg's res: 0.76191 + 0.0146164
[110]	cv_agg's res: 0.762851 + 0.0125565
[115]	cv_agg's res: 0.763102 + 0.0124766
[120]	cv_agg's res: 0.761662 + 0.0131573
[125]	cv_agg's res: 0.763872 + 0.0

{'res-mean': [0.6592739215263896,
  0.6733414517431037,
  0.691529502275969,
  0.7066655592554442,
  0.7207554026292394,
  0.7211897463065068,
  0.7323841028747267,
  0.7335006653523474,
  0.7319302137528073,
  0.7314289848618326,
  0.7344382320802875,
  0.7413687868244283,
  0.7456782393059331,
  0.7463391434338381,
  0.7458705879974398,
  0.7395350595550632,
  0.7398428702298675,
  0.741283828632381,
  0.7432870120895725,
  0.7440469414236309,
  0.7443135937870252,
  0.7459888148041934,
  0.7450769652319641,
  0.7442816040478996,
  0.7468105547974407,
  0.7469885613988723,
  0.7482488154862104,
  0.7503582815639694,
  0.7490351203672502,
  0.7519725159412847,
  0.7502937017557683,
  0.7522401252634819,
  0.7535513250481047,
  0.7546970278550393,
  0.7547356848398782,
  0.7555099164018896,
  0.7574709636337026,
  0.7586877447545124,
  0.7582940680007727,
  0.7587964920154805,
  0.7584840148425425,
  0.7593450389176253,
  0.7595995121023676,
  0.7615068869956602,
  0.7612135989810748,


In [17]:
model =lgb.train(lgb_params,dtrain,feval=evalMetric,verbose_eval=5,num_boost_round=300,valid_sets=[dtrain])

[5]	training's res: 0.875765
[10]	training's res: 0.901707
[15]	training's res: 0.915308
[20]	training's res: 0.923402
[25]	training's res: 0.92788
[30]	training's res: 0.938081
[35]	training's res: 0.944418
[40]	training's res: 0.952298
[45]	training's res: 0.957941
[50]	training's res: 0.964556
[55]	training's res: 0.971513
[60]	training's res: 0.978394
[65]	training's res: 0.982344
[70]	training's res: 0.985208
[75]	training's res: 0.990756
[80]	training's res: 0.992434
[85]	training's res: 0.995002
[90]	training's res: 0.996159
[95]	training's res: 0.997526
[100]	training's res: 0.999102
[105]	training's res: 0.999106
[110]	training's res: 0.999333
[115]	training's res: 0.999555
[120]	training's res: 0.999777
[125]	training's res: 1
[130]	training's res: 1
[135]	training's res: 1
[140]	training's res: 1
[145]	training's res: 1
[150]	training's res: 1
[155]	training's res: 1
[160]	training's res: 1
[165]	training's res: 1
[170]	training's res: 1
[175]	training's res: 1
[180]	trainin

In [18]:
pred=model.predict(test.drop(['uid'],axis=1))

In [19]:
res =pd.DataFrame({'uid':test.uid,'label':pred})

In [20]:
res=res.sort_values(by='label',ascending=False)
res.label=res.label.map(lambda x: 1 if x>=0.5 else 0)
res.label = res.label.map(lambda x: int(x))

In [21]:
res.to_csv('/home/ss/Desktop/code/3/result_F.csv',index=False,header=False,sep=',',columns=['uid','label'])